In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle
from collections import defaultdict
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import random
import sys
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import sklearn.preprocessing as pp

In [4]:
with open('data/artists.p', 'rb') as fp:
    artists = pickle.load(fp)
    
with open('data/genres.p', 'rb') as fp:
    genres = pickle.load(fp)

with open('data/genres_clean.p', 'rb') as fp:
    genres_clean = pickle.load(fp)
    
with open('data/artists_clean.p', 'rb') as fp:
    artists_clean = pickle.load(fp)

In [5]:
mvp_dict = dict(random.sample(artists.items(), 20000))

In [6]:
len(mvp_dict)

20000

In [7]:
artists['Grateful Dead']

['classic rock',
 'cosmic american',
 'country rock',
 'double drumming',
 'folk rock',
 'jam band',
 'psychedelic rock',
 'rock']

In [8]:
df_genres = pd.DataFrame(columns=genres_clean)

In [9]:
df_genres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 5071 entries, 432hz to zydeco
dtypes: object(5071)
memory usage: 0.0+ bytes


In [10]:
df_artists = pd.DataFrame(mvp_dict.keys(), columns=['artist'])

In [11]:
df_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  20000 non-null  object
dtypes: object(1)
memory usage: 156.4+ KB


In [12]:
df_artists.head()

,artist
0,Haviitz
1,AJ_Barracuda
2,Talulah Neira
3,St. Catherine's Child
4,Constantin DrÄghici


In [13]:
mvp_df = pd.concat([df_artists,df_genres])

In [14]:
mvp_df.head()

,artist,432hz,48g,5th wave emo,8d,a cappella,a3,aarhus indie,aberdeen indie,abstract,...,zimdancehall,zither,zohioliin duu,zolo,zouglou,zouk,zouk riddim,zurich indie,zxc,zydeco
0,Haviitz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AJ_Barracuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Talulah Neira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,St. Catherine's Child,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Constantin DrÄghici,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
mvp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Columns: 5072 entries, artist to zydeco
dtypes: object(5072)
memory usage: 774.1+ MB


In [16]:
#df_genres.head()

In [17]:
#for k in artists.keys():
#    temp_df = {'artist': k}
#    df_genres.append(temp_df, ignore_index=True)

In [18]:
for col in mvp_df.columns[1:]:
    mvp_df[col].values[:] = 0

In [19]:
#for idx,name in enumerate(mvp_df['artist']):
    #print(idx,name)
    #cols = [g for g in mvp_dict[name]]
    #print(cols)
    #mvp_df.loc[idx,[cols]] = 1

In [20]:
mvp_df.head()

,artist,432hz,48g,5th wave emo,8d,a cappella,a3,aarhus indie,aberdeen indie,abstract,...,zimdancehall,zither,zohioliin duu,zolo,zouglou,zouk,zouk riddim,zurich indie,zxc,zydeco
0,Haviitz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AJ_Barracuda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Talulah Neira,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,St. Catherine's Child,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Constantin DrÄghici,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#mvp_dict['Tatlo']

In [22]:
#mvp_df.loc[mvp_df["artist"] == "Tatlo", "432hz"] = 0

In [23]:
for index in mvp_df.index:
    if mvp_df.loc[index,'artist'] in mvp_dict.keys():
        band = mvp_df['artist'][index]
#       print(mvp_dict[band])
#       mvp_df.loc[index,mvp_df[mvp_dict[band]] == 0, mvp_dict[band]] = 1
        mvp_df.loc[mvp_df["artist"] == band, mvp_dict[band]] = 1

In [24]:
mvp_df.head()

,artist,432hz,48g,5th wave emo,8d,a cappella,a3,aarhus indie,aberdeen indie,abstract,...,zimdancehall,zither,zohioliin duu,zolo,zouglou,zouk,zouk riddim,zurich indie,zxc,zydeco
0,Haviitz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AJ_Barracuda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Talulah Neira,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,St. Catherine's Child,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Constantin DrÄghici,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#mvp_df['48g'] = np.where(mvp_df['432hz']==1, 1, 0)
#mvp_df.head()

In [26]:
#print(mvp_df.loc[mvp_df['artist'] == 'Tatlo']['black noise'].values)

In [27]:
#for artist in mvp_df['artist']:
#    print(artist)

In [28]:
#mvp_dict['Junkyard']

In [29]:
#print(mvp_df.loc[mvp_df['artist'] == 'Junkyard']['rock'].values)

In [30]:
for val in mvp_df['jam band']:
    if val == 1:
        print(val)

1
1
1
1
1
1
1
1
1


In [31]:
mvp_df.index.rename('artist_id', inplace=True)

In [32]:
array_cols = mvp_df.columns[1:]

array_cols[:5]

Index(['432hz', '48g', '5th wave emo', '8d', 'a cappella'], dtype='object')

In [33]:
df_for_array = mvp_df[array_cols]

In [34]:
#mvp_sparse = sparse.csr_matrix(mvp_array)

#similarities = cosine_similarity(mvp_sparse)
#print('pairwise dense output:\n {}\n'.format(similarities))

#also can output sparse matrices
#similarities_sparse = cosine_similarity(mvp_sparse,dense_output=False)
#print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [35]:
df_sparsed = df_for_array.astype(pd.SparseDtype("int"))

In [36]:
df_sparsed.head()

,432hz,48g,5th wave emo,8d,a cappella,a3,aarhus indie,aberdeen indie,abstract,abstract beats,...,zimdancehall,zither,zohioliin duu,zolo,zouglou,zouk,zouk riddim,zurich indie,zxc,zydeco
artist_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
for idx,val in enumerate(df_sparsed['jam band']):
    if val == 1:
        print(mvp_df.loc[idx, 'artist'])

Ghost Light
Karl Denson
Donna The Buffalo
Cabinet
JJ Grey & Mofro
Leftover Salmon
The Breakfast
Giant Panda Guerilla Dub Squad
The String Cheese Incident


In [38]:
mvp_array = df_sparsed.to_numpy()

In [39]:
# Change chunk_size to control resource consumption and speed
# Higher chunk_size means more memory/RAM needed but also faster 
#chunk_size = 100
#matrix_len = mvp_array.shape[0] # Not sparse numpy.ndarray

#def similarity_cosine_by_chunk(start, end):
#    if end > matrix_len:
#        end = matrix_len
#    return cosine_similarity(X=mvp_array[start:end], Y=mvp_array) # scikit-learn function

#for chunk_start in range(0, matrix_len, chunk_size):
#    cosine_similarity_chunk = similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size)
    # Handle cosine_similarity_chunk  ( Write it to file_timestamp and close the file )
    # Do not open the same file again or you may end up with out of memory after few chunks 

In [40]:
#len(cosine_similarity_chunk)

In [41]:
data_sparse = sparse.csr_matrix(df_sparsed)

In [42]:
# calculate similarity between each row (artist x genre)
similarities_sparse = cosine_similarity(data_sparse, dense_output=False)

In [43]:
# returns index (column position) of top n similarities in each row
def top_n_idx_sparse(matrix, n):
    '''Return index of top n values in each row of a sparse matrix'''
    top_n_idx = []
    for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
        n_row_pick = min(n, ri - le)
        top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]])
    return top_n_idx

artist_artist_similar = top_n_idx_sparse(similarities_sparse, 7)

In [44]:
# transforms result from sparse matrix into a dict as {matrix_index: [user_1, user_2,..., user_5]}
artist_similarity_dict = {}
for idx, val in enumerate(artist_artist_similar):
        artist_similarity_dict.update({idx: val.tolist()})

In [45]:
#artist_artist_similar
#artist_similarity_dict

In [46]:
# gets actual user ids from data based on sparse matrix position index
similar_artists_final = {}
for artist, similar_artists in artist_similarity_dict.items():
    idx = mvp_df.index[artist]
    values = []
    for value in similar_artists:
        values.append(mvp_df.index[value])

    similar_artists_final.update({idx: values})

In [47]:
count = 0

for i in similar_artists_final:
    print(i, similar_artists_final[i])
    count += 1
    
    if count > 10:
        break

0 [9963, 13942, 5254, 13101, 1269, 275, 0]
1 [17709, 16915, 8656, 5451, 1]
2 [6836, 13607, 16655, 4740, 3814, 3042, 2]
3 [18604, 3167, 4986, 6497, 7748, 11223, 3]
4 [2339, 4]
5 [2610, 6106, 2720, 16735, 1331, 11242, 5]
6 [13850, 17414, 10076, 6]
7 [17148, 12822, 10129, 7]
8 [16858, 13153, 12638, 16380, 4421, 2671, 8]
9 [19353, 14511, 18502, 14810, 15137, 15847, 9]
10 [919, 18604, 13735, 7748, 3167, 10, 11223]


In [48]:
mvp_df[mvp_df.index == 1]['artist']

artist_id
1    AJ_Barracuda
Name: artist, dtype: object

In [49]:
#mvp_df[mvp_df.index == 7712]['artist']
#mvp_df[mvp_df.index == 6573]['artist']
mvp_df[mvp_df.index == 4616]['artist']
#mvp_df[mvp_df.index == 4022]['artist']
#mvp_df[mvp_df.index == 3975]['artist']
#mvp_df[mvp_df.index == 1683]['artist']

artist_id
4616    Theresa Andersson
Name: artist, dtype: object

In [50]:
# transforms list of artists: [similar_artist1, similar_artist2] into list of artist: [artist1, artist2]
similar_bands = {}
for artist, similar_artists in similar_artists_final.items():
    # remove the artist itself from similar_artists (since cos_sim(artist_1, artist_1) is 1)
    try:
        del similar_artists[similar_artists.index(artist)]
    except:
        pass
     # get artist ids from list of similar artists.
    artists_rec = mvp_df[(mvp_df.index.isin(similar_artists))]
    if artists_rec.empty:
        continue
    artists_sample = artists_rec.sample(n=10, replace=True).index.values
    similar_bands.update({artist: list(set(artists_sample))})

In [51]:
count = 0

for i in similar_artists_final:
    print(i, similar_artists_final[i])
    count += 1
    
    if count > 10:
        break

0 [9963, 13942, 5254, 13101, 1269, 275]
1 [17709, 16915, 8656, 5451]
2 [6836, 13607, 16655, 4740, 3814, 3042]
3 [18604, 3167, 4986, 6497, 7748, 11223]
4 [2339]
5 [2610, 6106, 2720, 16735, 1331, 11242]
6 [13850, 17414, 10076]
7 [17148, 12822, 10129]
8 [16858, 13153, 12638, 16380, 4421, 2671]
9 [19353, 14511, 18502, 14810, 15137, 15847]
10 [919, 18604, 13735, 7748, 3167, 11223]


In [52]:
count = 0

for i in similar_bands:
    print(i, similar_bands[i])
    count += 1
    
    if count > 10:
        break

0 [275, 5254, 13942, 9963]
1 [8656, 16915, 5451, 17709]
2 [3042, 6836, 3814, 16655]
3 [6497, 7748, 11223, 4986, 3167]
4 [2339]
5 [2720, 2610, 11242, 16735]
6 [13850, 10076, 17414]
7 [10129, 17148, 12822]
8 [13153, 4421, 2671, 16858, 16380, 12638]
9 [18502, 15847, 14511, 19353, 14810]
10 [7748, 13735, 18604, 11223, 919, 3167]


In [53]:
# transform dictionary into list of tuples and save on DataFrame
artist_sim_tup = [(artist, sim_artist) for artist, similar_bands in similar_bands.items() for sim_artist in similar_bands]
artist_similarity_df = pd.DataFrame(artist_sim_tup, columns=['artist_id', 'similar_artist_id'])

In [54]:
artist_similarity_df

,artist_id,similar_artist_id
0,0,275
1,0,5254
2,0,13942
3,0,9963
4,1,8656
...,...,...
84317,19998,7659
84318,19998,18136
84319,19998,18264
84320,19998,13055


In [55]:
rec_batch = pd.merge(artist_similarity_df, mvp_df['artist'], on='artist_id', how='left')

In [56]:
rec_batch[rec_batch['artist_id']<=3]

,artist_id,similar_artist_id,artist
0,0,275,Haviitz
1,0,5254,Haviitz
2,0,13942,Haviitz
3,0,9963,Haviitz
4,1,8656,AJ_Barracuda
5,1,16915,AJ_Barracuda
6,1,5451,AJ_Barracuda
7,1,17709,AJ_Barracuda
8,2,3042,Talulah Neira
9,2,6836,Talulah Neira


In [57]:
artist_map_dict = {}

for idx,row in enumerate(mvp_df['artist']):
    #print(row)
    artist_map_dict[idx] = row

In [58]:
artist_map_dict[2418]

'BOUNTY & COCOA'

In [59]:
rec_batch['similar_artist'] = rec_batch['similar_artist_id']

In [60]:
rec_batch['similar_artist'] = rec_batch['similar_artist'].map(artist_map_dict)

In [61]:
rec_batch.head(40)

,artist_id,similar_artist_id,artist,similar_artist
0,0,275,Haviitz,Fontarabie
1,0,5254,Haviitz,Louize
2,0,13942,Haviitz,falcxne
3,0,9963,Haviitz,Red Mass
4,1,8656,AJ_Barracuda,Just Imagine Africa
5,1,16915,AJ_Barracuda,Malkizy
6,1,5451,AJ_Barracuda,Tokyo Sauce
7,1,17709,AJ_Barracuda,Razor Man
8,2,3042,Talulah Neira,Shamanes Crew
9,2,6836,Talulah Neira,Zona Ganjah


In [62]:
rec_batch[rec_batch['artist'] == 'Little Feat']

,artist_id,similar_artist_id,artist,similar_artist


In [63]:
rec_batch[rec_batch['artist'] == 'Jefferson Starship']

,artist_id,similar_artist_id,artist,similar_artist


In [64]:
rec_batch[rec_batch['artist'] == 'Bob Seger']

,artist_id,similar_artist_id,artist,similar_artist


In [65]:
rec_batch[rec_batch['artist'] == 'Nazareth']

,artist_id,similar_artist_id,artist,similar_artist


In [66]:
rec_batch = (rec_batch.set_index(['artist',rec_batch.groupby('artist').cumcount()])['similar_artist']
        .unstack()
        .add_prefix('similar artist #')
        .reset_index())

In [67]:
rec_batch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19711 entries, 0 to 19710
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             19711 non-null  object
 1   similar artist #0  19711 non-null  object
 2   similar artist #1  18735 non-null  object
 3   similar artist #2  17168 non-null  object
 4   similar artist #3  14793 non-null  object
 5   similar artist #4  10435 non-null  object
 6   similar artist #5  3392 non-null   object
 7   similar artist #6  88 non-null     object
dtypes: object(8)
memory usage: 1.2+ MB


In [90]:
rec_batch.head(50)

,artist,similar artist #0,similar artist #1,similar artist #2,similar artist #3,similar artist #4,similar artist #5,similar artist #6
0,!Sooks,Denzel,Floewe,Popzin,Lyrikal Busta,Tlale Makhene,NaN,NaN
1,#1 Pop Ballads,Nicole Tiffany,Standing in the Hall of Fame,No Stopping Us Right Now I Feel You,DJ Mega Hitz,Wonderful Tonight,Spark Productions,Adelle
2,#Tehran Band,Amirashkan Gholami,Az Shanbe,Kilometre23,Abjeez,NaN,NaN,NaN
3,$OHO BANI,Rill,YBRE,Jhony Kaze,Rrotzer,makko,NaN,NaN
4,$unnÂ¥ Foreign,Jag hungry,018,NaN,NaN,NaN,NaN,NaN
5,"'Attenooije!', SociÃ©tÃ© Musicale d'Oeteldonque",Alle Daoge Meej,RomeinÃ¨tnie,Tusseneusenlippe,De Lempkes,De Ontdekkertjes,NaN,NaN
6,'The Great Comet' Original Cast Ensemble,Mary Rodgers,The Original Broadway Cast Of 'The Color Purple',Betty Buckley,Michael James Scott,Everybody's Talking About Jamie Cast,NaN,NaN
7,..Ð Ð¡ÐÐÐ,ÐÐ¸ÑÐ¾Ð½Ñ 3000,ÐÑÑÑÐ¸Ð¹ Ð¡Ð°Ð¼ÑÐ¹ ÐÐµÐ½Ñ,"1,5 ÐºÐ³ Ð¾ÑÐ»Ð¸ÑÐ½Ð¾Ð³Ð¾ Ð¿ÑÑÐµ",bezna,"Skulls, Angels and Sluts",NaN,NaN
8,018,$unnÂ¥ Foreign,Jag hungry,NaN,NaN,NaN,NaN,NaN
9,09 Bas Kol,Chilik Frank,Shmuli Ungar,NaN,NaN,NaN,NaN,NaN


In [69]:
#rec_batch.to_csv('mvp_artist_similarity_df.csv', index=False)

In [99]:
test_index = rec_batch.index
test_artist = '13th Sky'
test_con = rec_batch['artist'] == test_artist
test_idx = test_index[test_con]
final_idx = test_idx[0]

output = rec_batch[['similar artist #1', 'similar artist #2', 'similar artist #3', 'similar artist #4',
                'similar artist #5', 'similar artist #6']].iloc[final_idx]

In [96]:
final_idx

31

In [100]:
type(output)

pandas.core.series.Series

In [101]:
print(output)

similar artist #1    Whispers In The Shadow
similar artist #2        The Mist Of Avalon
similar artist #3              Wayne Hussey
similar artist #4            Arts Of Erebus
similar artist #5                       NaN
similar artist #6                       NaN
Name: 31, dtype: object


In [112]:
viable_artists = set(a.lower() for a in list(rec_batch['artist']))

In [113]:
len(viable_artists)

19704

In [116]:
#viable_artists

In [111]:
viable_artists[945]

'andy pickett'

In [133]:
def find_similar_artists(seed_artist, df):
    indices = df.index
    condition = df['artist'] == seed_artist
    temp_idx = indices[condition]
    #print(temp_idx)
    idx = temp_idx[0]

    similar_artists = df[['similar artist #1', 'similar artist #2', 'similar artist #3', 'similar artist #4',
                'similar artist #5', 'similar artist #6']].iloc[idx]

    return similar_artists

In [134]:
testing = find_similar_artists('Andy Pickett', rec_batch)

In [135]:
print(testing)

similar artist #1    Corkscrew Nose Dive
similar artist #2            Steve Gnash
similar artist #3                    NaN
similar artist #4                    NaN
similar artist #5                    NaN
similar artist #6                    NaN
Name: 945, dtype: object


In [184]:
def get_user_band(df):
    viable_artists = set(a.lower() for a in list(df['artist']))
    user_band = input('Tell me a band or musician whose musical associations you would like to explore: ').lower()

    if user_band in viable_artists:
        return " ".join(word[0].upper()+word[1:] for word in user_band.split(" "))
    else:
        print("Sorry, I haven't heard of them!")

In [185]:
get_user_band(rec_batch)

Tell me a band or musician whose musical associations you would like to explore:  andy pickett


'Andy Pickett'

In [186]:
band = get_user_band(rec_batch)

Tell me a band or musician whose musical associations you would like to explore:  andy pickett


In [187]:
band

'Andy Pickett'

In [188]:
find_similar_artists(band, rec_batch)

similar artist #1    Corkscrew Nose Dive
similar artist #2            Steve Gnash
similar artist #3                    NaN
similar artist #4                    NaN
similar artist #5                    NaN
similar artist #6                    NaN
Name: 945, dtype: object

In [189]:
#viable_artists[945]

In [190]:
similar_artists = find_similar_artists(band, rec_batch)

In [191]:
similar_artists

similar artist #1    Corkscrew Nose Dive
similar artist #2            Steve Gnash
similar artist #3                    NaN
similar artist #4                    NaN
similar artist #5                    NaN
similar artist #6                    NaN
Name: 945, dtype: object